In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

from copy import deepcopy


#For random forest for time series:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

#For metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import load_series, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DPModel, DPModel2
from Logic import logic_bat, logic_DP, logic_actions
from Funcs_Logic_DP import (get_price, get_emissions, logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, 
                            logic_series_print, DP_stochastic, policy_rollout)

In [2]:
house = "h16"
prod = load_series("p",house,"prod_"+house,[24,48,168])
cons = load_series("c",house,"cons_"+house,[24,48,168])
cons

,cons_h16,cons_h16-24,cons_h16-48,cons_h16-168,Year,Month,Day,Weekday,Hour
2020-12-29 00:00:00,5.7,5.7,6.9,6.3,2020,12,29,1,0
2020-12-29 01:00:00,5.9,5.4,5.3,4.8,2020,12,29,1,1
2020-12-29 02:00:00,5.0,6.0,4.9,4.8,2020,12,29,1,2
2020-12-29 03:00:00,3.9,6.8,4.4,4.3,2020,12,29,1,3
2020-12-29 04:00:00,3.5,4.0,3.6,4.0,2020,12,29,1,4
...,...,...,...,...,...,...,...,...,...
2023-01-10 16:00:00,6.1,7.6,6.9,5.4,2023,1,10,1,16
2023-01-10 17:00:00,8.3,10.0,9.8,6.2,2023,1,10,1,17
2023-01-10 18:00:00,10.4,9.1,14.5,13.0,2023,1,10,1,18
2023-01-10 19:00:00,9.9,11.1,10.8,10.4,2023,1,10,1,19


In [3]:
merged = merge(house)
merged

,prod_h16,cons_h16,power_yield,SpotPriceDKK,CO2Emission
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000,60.0
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000,58.0
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001,65.0
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998,67.0
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001,76.0
...,...,...,...,...,...
2022-12-31 19:00:00,0.0,8.7,-8.7,498.329987,209.0
2022-12-31 20:00:00,0.0,6.9,-6.9,301.190002,224.0
2022-12-31 21:00:00,0.0,5.3,-5.3,110.730003,253.0
2022-12-31 22:00:00,0.0,5.6,-5.6,73.919998,272.0


In [4]:
N = 24*7
bat_capacity=13
stepsize=2
Start="2022-06-19 00:00:00"
End = pd.date_range(start=Start, periods=N+(24-stepsize), freq="h")[-1]

train_prod = 2*7*24
train_cons = 3000

BeforeProd = pd.date_range(end=Start, periods=train_prod+1, freq="h")[0]
BeforeCons = pd.date_range(end=Start, periods=train_cons+1, freq="h")[0]

print(f"Period: From {Start} to {End}")

Period: From 2022-06-19 00:00:00 to 2022-06-26 21:00:00


In [5]:
forecasterProd = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=8,max_features=3,n_estimators=100,n_jobs=4),
                 lags      = list(range(1,10+1,1))
                )
forecasterCons = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=8,max_features=4,n_estimators=100,n_jobs=4),
                 lags      = list(range(1,10+1,1))
                )

battery_no = Battery(max_capacity=bat_capacity,max_charge=0)
battery    = Battery(max_capacity=bat_capacity,max_charge=7)
DPbat      = Battery(max_capacity=bat_capacity,max_charge=7)
DP2bat     = Battery(max_capacity=bat_capacity,max_charge=7)

series_no_battery  = pd.DataFrame(columns=merged.columns)
series_battery     = pd.DataFrame(columns=merged.columns)
series_battery_DP  = pd.DataFrame(columns=merged.columns)
series_battery_DP2 = pd.DataFrame(columns=merged.columns)

preds = pd.DataFrame() 
for i in range(N//stepsize):#stepsize has to be even
    _, predProd = backtesting_forecaster(forecaster = forecasterProd,
                                         y          = prod.loc[BeforeProd:End].iloc[i*stepsize:train_prod+i*stepsize+24]['prod_'+house],
                                         exog       = prod.loc[BeforeProd:End].iloc[i*stepsize:train_prod+i*stepsize+24].drop(['prod_'+house], axis=1),
                                         initial_train_size = train_prod,
                                         steps      = 24,
                                         refit      = True,
                                         metric     = 'mean_squared_error',
                                         fixed_train_size   = True,
                                         verbose    = False
                                        )

    _, predCons = backtesting_forecaster(forecaster = forecasterCons,
                                         y          = cons.loc[BeforeCons:End].iloc[i*stepsize:train_cons+i*stepsize+24]['cons_'+house],
                                         exog       = cons.loc[BeforeCons:End].iloc[i*stepsize:train_cons+i*stepsize+24].drop(['cons_'+house], axis=1),
                                         initial_train_size = train_cons,
                                         steps      = 24,
                                         refit      = True,
                                         metric     = 'mean_squared_error',
                                         fixed_train_size   = True,
                                         verbose    = False
                                        )
    pred_i = pd.merge(predProd, predCons, how="outer", left_index=True,right_index=True)
    pred_i = pred_i.rename(columns={"pred_x":"prod_"+house,"pred_y":"cons_"+house})

    pred_i["prod_"+house]=np.array((pred_i["prod_"+house].to_numpy())*10,dtype=int)/10
    pred_i["cons_"+house]=np.array((pred_i["cons_"+house].to_numpy())*10,dtype=int)/10

    pred_i["power_yield"] = pred_i["prod_"+house] - pred_i["cons_"+house]
    
    values = pred_i["power_yield"].to_numpy()
    
    for j in range(len(values)):
        values[j] = f"{values[j]:.1f}"
    
    pred_i["power_yield"] = values
    
    
    pred_i["SpotPriceDKK"] = merged.loc[pred_i.index[0]:pred_i.index[-1]]["SpotPriceDKK"]
    pred_i["CO2Emission"] = merged.loc[pred_i.index[0]:pred_i.index[-1]]["CO2Emission"]
    
    merged_i = merged.loc[Start:End].iloc[i*stepsize:i*stepsize+24]
    print(f"i: {i}, From {merged_i.index[0]} to {merged_i.index[-1]}")
    
    temp_no_bat = pred_logic_rollout(merged_i, pred_i, deepcopy(battery_no), logic_bat)
    temp_bat = pred_logic_rollout(merged_i, pred_i, deepcopy(battery), logic_bat)

    DP = DPModel(24, pred_i.index[0], pred_i.index[-1], house, pred_i, deepcopy(DPbat))
    _, pi = DP_stochastic(DP)
    _, _, actions = policy_rollout(DP,pi=lambda x, k: pi[k][x],x0=DPbat.get_current_capacity())
    
    DP2 = DPModel2(24, pred_i.index[0], pred_i.index[-1], house, pred_i, deepcopy(DP2bat))
    _, pi2 = DP_stochastic(DP2)
    _, _, actions2 = policy_rollout(DP2,pi=lambda x, k: pi2[k][x],x0=DP2bat.get_current_capacity())
    
    series_no_battery_i  = action_rollout(merged_i.iloc[:stepsize], battery_no, temp_no_bat[:stepsize])
    series_battery_i     = action_rollout(merged_i.iloc[:stepsize], battery, temp_bat[:stepsize])
    series_battery_DP_i  = action_rollout(merged_i.iloc[:stepsize], DPbat, actions)
    series_battery_DP2_i = action_rollout(merged_i.iloc[:stepsize], DP2bat, actions2)
    
    
    preds=pd.concat([preds,pred_i[:stepsize]])
    series_no_battery  = pd.concat([series_no_battery,series_no_battery_i])
    series_battery     = pd.concat([series_battery,series_battery_i])
    series_battery_DP  = pd.concat([series_battery_DP,series_battery_DP_i])
    series_battery_DP2 = pd.concat([series_battery_DP2,series_battery_DP2_i])

series_no_battery["cost_cummulative"] = series_no_battery["cost"].cumsum(axis=0)
series_battery["cost_cummulative"] = series_battery["cost"].cumsum(axis=0) 
series_battery_DP["cost_cummulative"] = series_battery_DP["cost"].cumsum(axis=0)
series_battery_DP2["cost_cummulative"] = series_battery_DP2["cost"].cumsum(axis=0)
series_no_battery["emission_cummulative"] = series_no_battery["emission"].cumsum(axis=0)
series_battery["emission_cummulative"] = series_battery["emission"].cumsum(axis=0) 
series_battery_DP["emission_cummulative"] = series_battery_DP["emission"].cumsum(axis=0) 
series_battery_DP2["emission_cummulative"] = series_battery_DP2["emission"].cumsum(axis=0) 
preds

i: 0, From 2022-06-19 00:00:00 to 2022-06-19 23:00:00
12 

KeyboardInterrupt: 

In [ ]:
print_price_summary(series_no_battery)
logic_series_print(series_no_battery)

In [ ]:
print_price_summary(series_battery)
logic_series_print(series_battery)

In [ ]:
print_price_summary(series_battery_DP)
logic_series_print(series_battery_DP)

In [ ]:
print_price_summary(series_battery_DP2)
logic_series_print(series_battery_DP2)